In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/My \Drive/NLP

In [ ]:
pip install torch~=2.4.0 torch_xla[tpu]~=2.4.0 -f https://storage.googleapis.com/libtpu-releases/index.html

In [ ]:
!pip install textattack==0.3.7

In [ ]:
!pip install lime

In [ ]:
!pip install shap

In [ ]:
# Load libraries
import nltk
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.optim import AdamW
import torch_xla
import torch_xla.core.xla_model as xm
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
import random
import nltk
from lime.lime_text import LimeTextExplainer
import numpy as np
from lime.lime_text import LimeTextExplainer
import torch
from transformers import DistilBertTokenizer
import random
from nltk.corpus import wordnet
from transformers import BertTokenizer
import shap
nltk.download('wordnet')

In [ ]:
# Load the dataset
df = pd.read_csv('./Data/KaggleData.csv')

# Convert to lowercase, remove punctuation, extra spaces, URLs, mentions, and hashtags
df['tweet'] = df['tweet'].str.lower().replace(r'[^\w\s]', '', regex=True).replace(' {2,}', ' ', regex=True).replace('"', '')
df['tweet'] = df['tweet'].replace(r'http\S+|www.\S+|@\w+|#\w+', '', regex=True)

# Define Dataset class for tokenization and encoding
# 0 - hate speech, 1 - offensive language, 2 - neither as positive or negative
class TweetDataset(Dataset):
    def __init__(self, tweets, labels, tokenizer, max_len):
        self.tweets = tweets
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.tweets)

    def __getitem__(self, idx):
        tweet = str(self.tweets[idx])
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            tweet,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'tweet_text': tweet,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Preprocessing
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
MAX_LEN = 128
BATCH_SIZE = 32
EPOCHS = 5

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(df['tweet'], df['class'], test_size=0.3, stratify=df['class'], random_state=42)

# Reset index for train and test DataFrames
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Prepare DataLoaders
train_dataset = TweetDataset(X_train, y_train, tokenizer, MAX_LEN)
test_dataset = TweetDataset(X_test, y_test, tokenizer, MAX_LEN)
train_data_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Model setup
device = xm.xla_device()
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(set(y_train))).to(device)

# Optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for batch in train_data_loader:
          input_ids = batch["input_ids"].to(device)
          attention_mask = batch["attention_mask"].to(device)
          labels = batch["labels"].to(device)
          outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

          loss = outputs.loss
          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

          xm.optimizer_step(optimizer)
          scheduler.step()
          optimizer.zero_grad()
          xm.mark_step()
          epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {epoch_loss / len(train_data_loader)}")

    # Save Model
    torch.save(model, './Weights/KaggleDistilBERT.pth')

model = torch.load('./Weights/KaggleDistilBERT.pth')

# Evaluation
model.eval()
y_pred = []
y_true = []

with torch.no_grad():
    for batch in test_data_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        _, preds = torch.max(outputs.logits, dim=1)
        y_pred.extend(preds.detach().cpu().numpy().tolist())
        y_true.extend(labels.detach().cpu().numpy().tolist())
true_labels, predictions = np.asarray(y_true), np.asarray(y_pred)

# Calculate accuracy, precision, recall, F1-score, and confusion matrix
accuracy = np.mean(np.array(predictions) == np.array(true_labels))
precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')
conf_mat = confusion_matrix(true_labels, predictions)

print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1-score: ", f1_score)
print("Confusion Matrix:\n", conf_mat)

In [ ]:
word2index = {}
for tweet in df['tweet']:
    for word in tweet:
        if word not in word2index:
            word2index[word] = len(word2index)

# Wrapper for the TextCNN model
class BERTWrapper:
    def __init__(self, model):
        self.model = model

    def __call__(self, text_input_list):
        preds = []
        for text in text_input_list:
            input_tensor = tokenize_and_pad([text]).long()
            output = self.model(input_tensor.to(device))
            pred = torch.softmax(output, dim=1).squeeze().tolist()
            preds.append(pred)
        return np.array(preds)

def tokenize_and_pad(text_list):
    max_length = 50
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    tokens = [tokenizer.tokenize(text)[:max_length] for text in text_list]
    token_indices = np.zeros((len(tokens), max_length), dtype=int)
    for i, tweet in enumerate(tokens):
        for j, word in enumerate(tweet):
            if word in word2index:
                token_indices[i, j] = word2index[word]
    return torch.tensor(token_indices)

wrapped_model = BERTWrapper(model)
class_names = ['hate_speech', 'offensive_language', 'neither']

# Explainability with SHAP
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def shap_analysis(text, wrapped_model, class_names):
    # Define a masker
    masker = shap.maskers.Text(tokenizer)

    # Initialize explainer
    explainer = shap.Explainer(wrapped_model, masker=masker)

    # Explain prediction
    shap_values = explainer([text])

    return shap_values

df['tweet'] = df['tweet'].apply(lambda x: ' '.join(x))
text_to_explain = random.choice(df['tweet'])
print("Text to explain:", text_to_explain)
shap_results = shap_analysis(text_to_explain, wrapped_model, class_names)

# Get predicted label for the text
predicted_label = np.argmax(wrapped_model([text_to_explain])[0])

shap.plots.text(shap_results[0])

def calculate_doe(shap_values, predicted_label):
    feature_scores = [abs(value) for value in shap_values.values[0][:, predicted_label]]
    std_dev = np.std(feature_scores)
    significant_features = len([score for score in feature_scores if score > std_dev])
    return significant_features / len(feature_scores)

doe = calculate_doe(shap_results, predicted_label)
print("Degree of Explainability (DoE):", doe)

# Define number of samples for analysis
num_sam = 20

# Load dataset
def load_custom_dataset(path):
    df = pd.read_csv(path)
    df = df.dropna(subset=['tweet', 'class'])
    return df

# SHAP Analysis
def shap_analysis(text, wrapped_model, class_names):
    # Define a masker
    masker = shap.maskers.Text(tokenizer)

    # Initialize explainer
    explainer = shap.Explainer(wrapped_model, masker=masker)

    # Explain prediction
    shap_values = explainer([text])

    return shap_values

# Calculate Degree of Explainability (DoE)
def calculate_doe(shap_values, predicted_label):
    feature_scores = [abs(value) for value in shap_values.values[0][:, predicted_label]]
    std_dev = np.std(feature_scores)
    significant_features = len([score for score in feature_scores if score > std_dev])
    return significant_features / len(feature_scores)

# Calculate average DoE for multiple samples
def calculate_average_doe(df, wrapped_model, class_names, samples=num_sam):
    doe_values = []
    sample_texts = random.sample(list(df['tweet']), samples)
    for text in sample_texts:
        shap_results = shap_analysis(text, wrapped_model, class_names)
        predicted_label = np.argmax(wrapped_model([text])[0])
        doe = calculate_doe(shap_results, predicted_label)
        doe_values.append(doe)
    average_doe = np.mean(doe_values)
    return average_doe

# Path to the dataset
test_data_path = "./Data/KaggleData.csv"
df = load_custom_dataset(test_data_path)

# Assuming wrapped_model and class_names are defined elsewhere
class_names = ['Hate speech', 'Offensive language', 'Neutral']

# Calculate and print average DoE
average_doe = calculate_average_doe(df, wrapped_model, class_names)
print(f"Average Degree of Explainability (DoE) for {num_sam} samples:", average_doe)